# ==============================================
# 📚 AI BOOK RECOMMENDATION CHATBOT (RAG)
# ==============================================
# ✅ Connects to Pinecone to retrieve stored book embeddings
# ✅ Converts user queries into embeddings
# ✅ Performs a vector similarity search in Pinecone
# ✅ Retrieves metadata for the top recommended books
# ✅ Uses Ollama to generate a conversational response
# ==============================================

In [1]:
# Import necessary libraries
import pinecone  # For vector search
from sentence_transformers import SentenceTransformer  # For text embeddings
import ollama  # For AI-generated responses

c:\Users\shuchismita_mallick.Shuchismita\anaconda3\envs\aienv\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
# -----------------------------------------------
# 🔹 STEP 1: INITIALIZE PINECONE CONNECTION
# -----------------------------------------------

# Pinecone API credentials
PINECONE_API_KEY = "pcsk_6WjJ7w_Mdge2WY3AVCqN5RafTo3byPqv7ECMdRSr1SZ4oS6quNbkDb4Lc7tnE2esM9DKQe"
PINECONE_ENV = "us-east-1"  # Your selected region

# Initialize Pinecone
pinecone_client = pinecone.Pinecone(api_key=PINECONE_API_KEY, environment=PINECONE_ENV)

# Connect to your Pinecone index
index_name = "book-recommendations"
index = pinecone_client.Index(index_name)

In [3]:
# -----------------------------------------------
# 🔹 STEP 2: LOAD THE EMBEDDING MODEL
# -----------------------------------------------
# Load the sentence-transformer model (same as the one used for storing book embeddings)
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

def get_query_embedding(query):
    """
    Converts a user query into an embedding using the pre-trained sentence transformer model.
    
    Args:
        query (str): The user's search query.
    
    Returns:
        list: A vector embedding of the query text.
    """
    return embedding_model.encode(query).tolist()  # Convert to list for Pinecone search


In [4]:
# -----------------------------------------------
# 🔹 STEP 3: SEARCH PINECONE FOR SIMILAR BOOKS
# -----------------------------------------------
def retrieve_similar_books(user_query, top_k=5):
    """
    Converts a user query into an embedding and retrieves the most similar books from Pinecone.
    
    Args:
        user_query (str): The user's search request.
        top_k (int): The number of recommended books to retrieve.
    
    Returns:
        list: A list of recommended books with metadata (title, author, genre, similarity score).
    """
    # Convert user query into an embedding
    query_embedding = get_query_embedding(user_query)

    # Perform a vector similarity search in Pinecone
    search_results = index.query(vector=query_embedding, top_k=top_k, include_metadata=True)

    # Extract book metadata from search results
    recommended_books = []
    for match in search_results["matches"]:
        recommended_books.append({
            "title": match["metadata"].get("title", "Unknown Title"),
            "author": match["metadata"].get("author", "Unknown Author"),
            "genre": match["metadata"].get("category", "Unknown Category"),
            "score": match["score"]  # Similarity score
        })

    return recommended_books


In [5]:
# -----------------------------------------------
# 🔹 STEP 4: FORMAT RESULTS & GENERATE RESPONSE USING OLLAMA
# -----------------------------------------------
def generate_recommendation_response(user_query, recommended_books):
    """
    Takes the user query and retrieved book recommendations, and generates a conversational response using Ollama.
    
    Args:
        user_query (str): The user's search query.
        recommended_books (list): A list of recommended books with metadata.
    
    Returns:
        str: A user-friendly, AI-generated response summarizing the book recommendations.
    """
    # Format the recommended books into a structured text
    book_list = "\n".join([
        f"📖 Title: {book['title']}\n"
        f"👨‍💻 Author: {book['author']}\n"
        f"🏷️ Genre: {book['genre']}\n"
        f"⭐ Relevance Score: {book['score']:.2f}\n"
        for book in recommended_books
    ])

    # Construct the prompt for Ollama
    prompt = f"""
    You are an AI book expert helping a user find the best books.

    User Query: "{user_query}"

    Based on their request, here are some book recommendations:

    {book_list}

    Please generate a friendly, engaging response that summarizes these book recommendations in a conversational way.
    """

    # Query Ollama with the formatted prompt
    response = ollama.chat(model="mistral", messages=[{"role": "user", "content": prompt}])

    return response["message"]["content"]  # Extract the generated response


In [6]:
# -----------------------------------------------
# 🔹 STEP 5: MAIN FUNCTION TO RUN THE RECOMMENDATION SYSTEM
# -----------------------------------------------
def recommend_books(user_query, top_k=5):
    """
    Retrieves similar books from Pinecone and generates a conversational response using Ollama.
    
    Args:
        user_query (str): The user's search request.
        top_k (int): The number of recommendations to fetch.
    
    Returns:
        str: A natural language response with book recommendations.
    """
    # Retrieve similar books from Pinecone
    recommended_books = retrieve_similar_books(user_query, top_k)

    # If no recommendations found, return a message
    if not recommended_books:
        return "❌ Sorry, I couldn't find any matching books. Try a different genre or theme!"

    # Generate a user-friendly response using Ollama
    response = generate_recommendation_response(user_query, recommended_books)

    return response


In [7]:
# -----------------------------------------------
# 🔹 STEP 6: TESTING THE RECOMMENDATION SYSTEM
# -----------------------------------------------
# Example query
user_query = "A mystery novel with a detective solving a crime"

# Get book recommendations
response = recommend_books(user_query)

# Print the AI-generated response
print("\n📚 AI Book Recommendation:\n")
print(response)



📚 AI Book Recommendation:

 Hi there! I'm delighted to help you find the perfect mystery novel with a detective solving a crime. Here are some recommendations based on your request:

1. "Crime Novels" - This is an anthology of short stories by various authors, all revolving around the theme of crime and detection. It's a great way to explore different writing styles and plot twists!

2. "The Simple Art of Murder" by Raymond Chandler - A classic detective novel set in post-war America. The story follows Philip Marlowe as he investigates a seemingly simple case that leads to a complex web of deceit and danger.

3. "The Hidden Hand" by Carroll John Daly - This is an early detective novel featuring the character John Quill, who's tasked with solving a series of baffling murders in a remote mining town. The plot is filled with suspense and surprise!

4. "Filth" by Irvine Welsh - While it might not be your traditional detective story, it does feature a detective protagonist struggling with 